## Professional Console Apps using Click

In [1]:
import click

@click.command()
def main():
    print("I'm a beautiful CLI ✨")

main()

Usage: ipykernel_launcher.py [OPTIONS]
Try "ipykernel_launcher.py --help" for help.

Error: no such option: -f


SystemExit: 2

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
import requests 

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)

    return response.json()

current_weather('Lisbon')

{'coord': {'lon': -0.13, 'lat': 51.51},
 'weather': [{'id': 300,
   'main': 'Drizzle',
   'description': 'light intensity drizzle',
   'icon': '09d'}],
 'base': 'stations',
 'main': {'temp': 280.32,
  'pressure': 1012,
  'humidity': 81,
  'temp_min': 279.15,
  'temp_max': 281.15},
 'visibility': 10000,
 'wind': {'speed': 4.1, 'deg': 80},
 'clouds': {'all': 90},
 'dt': 1485789600,
 'sys': {'type': 1,
  'id': 5091,
  'message': 0.0103,
  'country': 'GB',
  'sunrise': 1485762037,
  'sunset': 1485794875},
 'id': 2643743,
 'name': 'London',
 'cod': 200}

### Adding a mandatory parameter to click

In [ ]:
import click
import requests

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    # we can immediatly access a specific value inside the json response
    return response.json()['weather'][0]['description']

@click.command()
@click.argument('location')
def main(location):
    weather = current_weather(location)
    print(f"The weather in {location} right now: {weather}.")
    
main()

# in a terminal call: python cli.py lisbon

### Adding the `api_key` as an optional parameter to click

In [ ]:
import click
import requests

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    return response.json()['weather'][0]['description']

@click.command()
@click.argument('location')
# the optional parameter
# click creates the option passed to the main function by stripping the leading dashes and turning them into snake case. --api-key becomes api_key.
@click.option('--api-key', '-a')
def main(location, api_key):
    weather = current_weather(location, api_key)
    print(f"The weather in {location} right now: {weather}.")
    
main()

# in a terminal call: python cli.py london --api-key b1b15e88fa797225412429c1c50c122a1

### Adding auto-generated usage instructions

If you run `python cli.py --help` you'll get the following:

````
Usage: cli.py [OPTIONS] LOCATION

Options:
  -a, --api-key TEXT
  --help              Show this message and exit.
````

Let's add some more documentation:

In [ ]:
import click
import requests

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    return response.json()['weather'][0]['description']

@click.command()
@click.argument('location')
# add a description to the option
@click.option(
    '--api-key', '-a',
    help='Your API key for the OpenWeatherMap API',
)
def main(location, api_key):
    # add a general description to our application
    """
    A little weather tool that shows you the current weather in a LOCATION of
    your choice. Provide the city name and optionally a two-digit country code.
    Here are two examples:

    1. London,UK

    2. Lisbon

    You need a valid API key from OpenWeatherMap for the tool to work. You can
    sign up for a free account at https://openweathermap.org/appid.
    """
    weather = current_weather(location, api_key)
    print(f"The weather in {location} right now: {weather}.")
    
main()

#run python cli.py --help

### Adding more commands (called sub-commands)

For this to be possible, we have to change the structure of our application: the entry point of our application will stop being a command and will now be a group of commands.

In [ ]:
import click
import requests

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    return response.json()['weather'][0]['description']

# main() is now a group of sub-commands
@click.group()
def main():
    pass

# one of main's sub-command is now 'current'
@main.command()
@click.argument('location')
@click.option(
    '--api-key', '-a',
    help='Your API key for the OpenWeatherMap API',
)
def current(location, api_key):
    """
    A little weather tool that shows you the current weather in a LOCATION of
    your choice. Provide the city name and optionally a two-digit country code.
    Here are two examples:

    1. London,UK

    2. Lisbon

    You need a valid API key from OpenWeatherMap for the tool to work. You can
    sign up for a free account at https://openweathermap.org/appid.
    """
    weather = current_weather(location, api_key)
    print(f"The weather in {location} right now: {weather}.")
    
main()

# run python cli.py current lisbon --api-key b1b15e88fa797225412429c1c50c122a1

### Storing the API key in a configuration file using another sub-command

What we want is a way to store an API key in a configuration file, using a separate command. Let's call it `config` and make it ask the user to enter their API key.

In [ ]:
import click
import requests

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    return response.json()['weather'][0]['description']

@click.group()
def main():
    pass

@main.command()
@click.argument('location')
@click.option(
    '--api-key', '-a',
    help='Your API key for the OpenWeatherMap API',
)
def current(location, api_key):
    """
    A little weather tool that shows you the current weather in a LOCATION of
    your choice. Provide the city name and optionally a two-digit country code.
    Here are two examples:

    1. London,UK

    2. Lisbon

    You need a valid API key from OpenWeatherMap for the tool to work. You can
    sign up for a free account at https://openweathermap.org/appid.
    """
    weather = current_weather(location, api_key)
    print(f"The weather in {location} right now: {weather}.")
    
# the new config sub-command
@main.command()
def config():
    """
    Store configuration values in a file.
    """
    print("I handle the configuration.")
    
main()

# run python cli.py config

#### Asking the user for command-line input

In [ ]:
import click
import requests
# import the os package to access the filesystem
import os

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    return response.json()['weather'][0]['description']

@click.group()
def main():
    pass

@main.command()
@click.argument('location')
@click.option(
    '--api-key', '-a',
    help='Your API key for the OpenWeatherMap API',
)
def current(location, api_key):
    """
    A little weather tool that shows you the current weather in a LOCATION of
    your choice. Provide the city name and optionally a two-digit country code.
    Here are two examples:

    1. London,UK

    2. Lisbon

    You need a valid API key from OpenWeatherMap for the tool to work. You can
    sign up for a free account at https://openweathermap.org/appid.
    """
    weather = current_weather(location, api_key)
    print(f"The weather in {location} right now: {weather}.")
    
@main.command()
# add the option to the sub-command
@click.option(
    '--api-key', '-a',
    help='your API key for the OpenWeatherMap API',
)
# add the option as an argument to the sub-command's function
def config(api_key):
    """
    Store configuration values in a file.
    """
    # define the path to the configuration file
    config_file = f'{os.getcwd()}/settings.cfg'
    
    # ask the user for the value of the setting
    api_key = click.prompt(
        "Please enter your API key",
        default=api_key
    )

    # write the setting to the file
    with open(config_file, 'w') as cfg:
        cfg.write(api_key)
    
main()

# run python cli.py config
# run python cli.py config --api-key b1b15e88fa797225412429c1c50c122a1

### Handling user input

### Enforcing option types

You can specify the accepted data type of an option, like this (see [here](https://click.palletsprojects.com/en/6.x/parameters/#parameter-types) for all the possibilities):

`@click.option('--api-key', '-a', type=str)`

But our api key, although of a specific type (str), has a special format: it's a 32 hexadecimal string, meaning it always has a length of 32 and contains numbers 0 to 9 and letters between a and f. So we have to build a custom type.

#### Building a custom option type

We can take advantage of all the code that the other options already have and just change what we want by creating our new option as an instance of the `ParamType` class in Click:

In [ ]:
class ApiKey(click.ParamType):

    def convert(self, value, param, ctx):
        return value

What are those variables in the function?

* `value` - the user’s input
* `param` - will contain the option that we declared using the `click.option` or `click.argument` decorators
* `ctx` - the context of the command. Ignore for now, it's not important.

This is the new code:

In [ ]:
# since we'll be using regular expressions to determine the format of our string, we need the re package
import re

class ApiKey(click.ParamType):
    name = 'api-key'

    def convert(self, value, param, ctx):
        found = re.match(r'[0-9a-f]{32}', value)

        if not found:
            self.fail(
                f'{value} is not a 32-character hexadecimal string',
                param,
                ctx,
            )

        return value

All we have to do now is plug this new parameter type into our existing `config` command.

In [ ]:
import click
import requests
import os
# since we'll be using regular expressions to determine the format of our string, we need the re package
import re

SAMPLE_API_KEY = 'b1b15e88fa797225412429c1c50c122a1'

def current_weather(location, api_key=SAMPLE_API_KEY):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    return response.json()['weather'][0]['description']

# our new custom option validator
class ApiKey(click.ParamType):
    name = 'api-key'

    def convert(self, value, param, ctx):
        found = re.match(r'[0-9a-f]{32}', value)

        if not found:
            self.fail(
                f'{value} is not a 32-character hexadecimal string',
                param,
                ctx,
            )

        return value

@click.group()
def main():
    pass

@main.command()
@click.argument('location')
@click.option(
    '--api-key', '-a',
    help='Your API key for the OpenWeatherMap API',
)
def current(location, api_key):
    """
    A little weather tool that shows you the current weather in a LOCATION of
    your choice. Provide the city name and optionally a two-digit country code.
    Here are two examples:

    1. London,UK

    2. Lisbon

    You need a valid API key from OpenWeatherMap for the tool to work. You can
    sign up for a free account at https://openweathermap.org/appid.
    """
    weather = current_weather(location, api_key)
    print(f"The weather in {location} right now: {weather}.")
    
@main.command()
@click.option(
    '--api-key', '-a',
    # plug our custom option validator
    type=ApiKey(),
    help='Your API key for the OpenWeatherMap API',
)
def config(api_key):
    """
    Store configuration values in a file.
    """
    
    config_file = f'{os.getcwd()}/settings.cfg'
    
    api_key = click.prompt(
        "Please enter your API key",
        default=api_key
    )

    with open(config_file, 'w') as cfg:
        cfg.write(api_key)
    
main()

# run python cli.py config --api-key invalid

### Using the context to pass parameters between commands

You probably thought about the command we created, our new API key option and wondered if this means we actually have to define the option on both of our commands, `config` and `current`. And your assumption would be correct.

How can we avoid defining the same option on both commands? We use a feature called the “Context”. Click executes every command within a context that carries the definition of the command as well as the input provided by the user. And it comes with a placeholder object called obj, that we can use to pass arbitrary data around between commands.

First let’s look at our group and how we can get access to the context of our main entrypoint:

In [ ]:
@click.group()
# we want access to the context of the command (or group)
@click.pass_context
# we're putting that context in the ctx variable
def main(ctx):
    ctx.obj = {}

Let's insert our API key in the context to make it available to all sub-commands:

In [ ]:
@click.group()
@click.option(
    '--api-key', '-a',
    type=ApiKey(),
    help='Your API key for the OpenWeatherMap API',
)
@click.pass_context
def main(ctx, api_key):
    ctx.obj = {
        'api_key': api_key,
    }

With the API key stored in the context, we can now get access to it in both of our sub-commands by adding the `pass_context` decorator as well:

In [ ]:
@main.command()
@click.pass_context
def config(ctx):
    api_key = ctx.obj['api_key']

### Wrapping up

Here's the final code:

In [ ]:
import click
import requests
import os
import re

def current_weather(location, api_key):
    url = 'http://samples.openweathermap.org/data/2.5/weather'

    query_params = {
        'q': location,
        'appid': api_key,
    }

    response = requests.get(url, params=query_params)
    
    return response.json()['weather'][0]['description']

class ApiKey(click.ParamType):
    name = 'api-key'

    def convert(self, value, param, ctx):
        found = re.match(r'[0-9a-f]{32}', value)

        if not found:
            self.fail(
                f'{value} is not a 32-character hexadecimal string',
                param,
                ctx,
            )

        return value

@click.group()
@click.option(
    '--api-key', '-a',
    type=ApiKey(),
    help='Your API key for the OpenWeatherMap API',
)
@click.option(
    '--config-file', '-c',
    type=click.Path(),
    default=f'{os.getcwd()}/settings.cfg',
)
@click.pass_context
def main(ctx, api_key, config_file):
    if not api_key and os.path.exists(config_file):
        with open(config_file) as cfg:
            api_key = cfg.read()

    ctx.obj = {
        'api_key': api_key,
        'config_file': config_file,
    }

@main.command()
@click.argument('location')
@click.pass_context
def current(ctx, location):
    """
    Show the current weather for a location using OpenWeatherMap data.
    """
    api_key = ctx.obj['api_key']
    
    if not api_key:
        click.secho('Missing API key. Run the config command or use the -a flag to specify a key.', fg='red')
        exit()
    
        weather = current_weather(location, api_key)
        print(f"The weather in {location} right now: {weather}.")
    
@main.command()
@click.pass_context
def config(ctx):
    """
    Store configuration values in a file, e.g. the API key for OpenWeatherMap.
    """
    config_file = ctx.obj['config_file']

    api_key = click.prompt(
        "Please enter your API key",
        default=ctx.obj.get('api_key', '')
    )
    
    if not re.match(r'[0-9a-f]{32}', api_key):
        click.echo(click.style(f'{api_key} is not a 32-character hexadecimal string.', fg='red'))
        exit()

    with open(config_file, 'w') as cfg:
        cfg.write(api_key)
    
main()

# Perform the following tests:

# delete the settings.cfg file if exists
# run python cli.py
# run python cli.py conf
# run python cli.py --co
# run python cli.py config and paste: invalid
# run python cli.py config and paste: b1b15e88fa797225412429c1c50c122a1
# run python cli.py config and check if there's a default value now
# run python cli.py current
# run python cli.py current lisbon